In [ ]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
from pandas.api.types import CategoricalDtype
from scipy.sparse.linalg import norm
from tqdm import tqdm_notebook as tqdm

# import self-developed functions
from doc_function import *

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/test")
    sys.path.append(module_path+"/")

import RecEval
re = RecEval.RecEval()

# FS20C7 - Recommendations in Retail
#### Documentation
----
Authors: Simon Luder, Roman Studer

## Table of Content
- Management Summary
    - Initial Situation
    - Procedure
    - Result
- 1. Introduction
- 2. Theoretical Background
    - 2.1 Main goal
    - 2.2 Recommender System Algorithms
    - 2.3 Evaluation options
- 3. Methods
    - 3.1 Data reduction
    - 3.2 Ratings
    - 3.3 Interaction Matrix
    - 3.4 Similarity Matrix
    - 3.5 Prediction Matrix
    - 3.6 Implementation
- 4. Results
    - 4.1 Metrics per Algorithm
    - 4.2 Comparison of the results
- 5. Discussion
- 6. Conclusion and Outlook
- 7. Sources
- 8. Glossary

## Management Summary

### Initial Situation
The scientific report aims to present the essential aspects of a recommender system down to the mathematical basics. The individual steps from the creation of an interaction matrix, similarity matrix to the prediction of a rating for items that a customer has not yet purchased are included. The basis is a dataset which contains transactions of customers. For each transaction information of User Id, Product Id, Order Id, Department of the product, Aisle of the product and if the product has been ordered before are available.

### Procedure
A recommender can be created in several ways. There are two basic procedures. Model-based and memory-based. This document focuses on memory based recommenders which predicts a rating for products using an interaction and similarity matrix. Memory based recommenders can be further divided into two categories. Item based collaborative filtering (IBCF) and user based collaborative filtering (UBCF). The procedure is the same for both. With the help of the list of transactions an interaction matrix is created which records the interactions (in our case orders) for each item for each user. On this basis, a similarity matrix is created which records the similarity of two products in a numerical value. With the interaction matrix and the similarity matrix a prediction matrix can be created, which has basically the same structure as an interaction matrix, but predicts a rating for each user for each product not yet purchased. Based on this prediction, products can then be recommended. Using various metrics, mathematical and heuristic, a recommender is then evaluated. Different methods for calculating the individual matrices can be applied. A comparison is given at the end of the report or under Result.

### Result
<span style="color:blue">What are our deliverables?</span>

----
## 1 Introduction
This scientific report serves to compare different methods of evaluating an Item based Collaborative Filtering (IBCF) recommender. To test different methods against each other, a plug & play recommender system was created from scratch (with minimal use of existing libraries). From product reduction to the evaluation method. This system allows you to quickly test different methods and compare the results. The report first explains the basic idea of a recommender and shows the functions implemented in the Plug & Play recommender.

As a basis we use a dataset from the "[Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/data)" (Instacart Market Basket Analysis n.d.) challenge on Kaggle. This dataset contains several million orders of numerous products from thousands of users. 

The official assignment was prepared by lecturers from the University of Applied Sciences Northwestern Switzerland. Further information can be found at the following address: https://ds-spaces.technik.fhnw.ch/fs20c7/ (Requires login information)

The following delivery objects are agreed with the client:

 - Written concept for the planned procedure, in particular cleaning up the product shelf and defining the success criteria
 - Documentation (possibly presentation) of the implemented recommender system with explanation of the tested algorithms, the methodical approach and the gained insights (incl. Q&A)
 - Comparison and assessment of the model quality of different model- and memory-based recommender systems with (a) methods of statistical off-line evaluation and (b) assessment of the personalisation of proposals
 - Documentation of the methods, analyses and results of steps 2 and 3 in the form of a scientific report. 
 - Reflection of the scientific procedure by means of a criteria sheet (basis delivery objects 1-4)

In a first step, the assignment is to implement a simple model- or memory-based recommender system itself "from scratch", i.e. without using packages, and to evaluate the model quality "off-line". This should create a small experimental field in which different algorithms can be systematically tested and compared. The knowledge gained about the functionality and programming techniques should be used to make a well-founded selection from solutions offered in packages (see point 2 above). 

In a further step, packages will be used to make further systematic comparisons of several recommender system approaches and then to quantify and visualise the degree of personalisation of the sales recommendations made (see point 3 above). 

The findings from the Challenge are to be documented in a systematic form as a scientific report (point 4 above), which is generated as a PDF from the Code Notebook. Furthermore, the scientific procedure is to be reflected on the basis of a criteria sheet (point 5 above). 

----
## 2 Theoretical Background 

>In a very general way, recommender systems are algorithms aimed at suggesting relevant items to users (items being movies to watch, text to read, products to buy or anything else depending on industries). (Rocca 2019)

A recommender is used to suggest products that a user has not yet interacted with, based on their personal preferences. The recommender learns these preferences by orienting himself on the previous interactions of the user and with the help of which he creates a user profile (Model based recommender) or searches for similar users and items (Memory based recommender). Recommenders are best known in online shops. Amazon shows products that 'may interest you' or Netflix suggests movies that you 'might like' based on your personal interests. These systems are of great importance to help customers find what they are looking for as quickly as possible. With a huge product shelf like Amazon has, a customer is dependent on it because it is impossible for them to find products they are not familiar with. There are two distinctions here. Firstly, a recommender can help the customer to discover new things and show them new, unknown preferences. The recommender thus helps the customer to 'explore'. Second, a recommender can also 'exploit' the known preferences of users to recommend products that match the known preferences.  

### 2.1 Main goal
<span style="color:blue">Explain what we want to build. A modular recommender to explore diffrent variants and combinations of recommder systems.</span>
The aim of this report is to compare different compilations of recommenders with each other and to get a better understanding of the structure of a recommender and the impact of different methods on its usability. For this reason, we build a modular recommender from scratch, which includes several methods to create a rating, several methods to calculate a similarity and several methods to create predictions and evaluate the recommender. This allows us to get several different recommendations from the same dataset input. 

### 2.2 Recommender System Algorithms
<span style="color:blue">This section deals with the different approaches for recommender systems. E.g collaborative filtering, content based, hybrid approach.
</span>

#### 2.2.1 User and Item based collaborative filtering

<img src='./images/flow_ibcf.png' width='90%'>

Collaborative filtering uses different methods to calculate the similarity between two products or two users. In an item-based approach, a product is compared with another by comparing it to another product. The more similar the interactions of customers between these two products are, the more they fit together. 
With the user based approach the same happens, but instead of products, customers are compared with each other. 

A recommender using collaborative filtering needs a certain number of interactions per customer to deliver useful results. Therefore it makes sense to offer generic recommendations to users who have not yet often interacted with the products. Since the preferences of the customer cannot yet be determined exactly. Here a list of "Top Products" or "Most Purchased" is useful. In the **reduction** phase it makes sense to filter out customers with less than n orders and to make generic recommendations to them. The customers with more than n orders are suitable for further processing. The transactions of these customers can be recorded in an **interaction matrix** $I$ with the dimension $user \times item$. The value for $I_{u,i}$ represents the rating of the customer $u$ for item $i$. This rating can be recorded in various forms: 

- A rating can be a simple unary rating which only states if the user $u$ has interacted with the item $i$ at some point. This binary rating can also be applied if the user provides a binary rating. A like button, for example, provides the same unary/binary rating if a transaction is recorded not by buying something but by liking a movie, an image or something similar. 

$$\displaystyle{ r(u,i) =
  \begin{cases}
    1 & \quad \text{if } u \text{ interacted with item } i\\
    0 & \quad \text{otherwise}\\
  \end{cases}}$$

- Another rating counts the number of interactions per user $u$ and item $i$. This provides an ordinal rating. The rating could for example record the number of orders for the same product by the same user:

$$r(u,i) = \sum_{i = 1}^n 1 \text{  , for n of interactions}$$

- If the transaction records a rating that the customer provided himself, like a rating on a scale from 1 to 10. This provided rating can be used to fill $I_{u,i}$ without any calculation. Something to keep in mind using this approach is that a rating 0 equals "no interaction":

$$\displaystyle{ r(u,i) =
  \begin{cases}
    r & \quad \text{if } u \text{ provided a rating for } i\\
    \text{0} & \quad \text{otherwise}\\
  \end{cases}}$$

We have also designed an own rating which gets introduced in section 3.2.3

The next step is to use the interaction matrix $I$ and a similarity function, like "cosine similarity", to calculate the similarity between each item or each user. The resulting **Similarity Matrix** $S$ therefore has the dimensions $user \times user$ or $item \times item$. An item-item matrix defines the collaborative filter as item-based collaborative filtering (IBCF) and a user-user matrix defines it as user-based collaborative filtering (UBCF) because the first is looking for similar items and the second for similar users. Not every similarity function is applicable to a UBCF _and_ an IBCF. For example, Pearson correlation is best suited for user-users, i.e. a UBCF, where Cosine similarity gives the best results for item-item, i.e. an IBCF. For example, if we work with an IBCF, we can use the similarity function cosine similarity to determine the similarity $s$ of two items. Cosine similarity works with the intermediate angle function and thus calculates the similarity $s$ vectorized. Since the resulting matrix $S$ is symmetric only the upper or lower triangular matrix has to be calculated: (Three of several methods are described in detail in section 3.4.) 

$$
s(i,j) = \frac{r_{i}\cdot r_{j}}{\|r_{i}\|_{2}\cdot\|r_{j}\|_{2}}
$$

In a next step the similarity matrix can be used to predict ratings for items with which a user has not yet interacted using a prediction function. This prediction matrix $P$ has the same dimensions as the interaction matrix $I$, i. e. $user \times item$. Since the prediction function predicts a rating for each product, this matrix $P$ is necessarily a dense matrix. The information from the prediction matrix can then be used to suggest new items to a user. It is also possible to recommend items directly from the similarity matrix $S$. A KNN algorithm searches for the items that best match each other. Please note that it does not make sense to suggest items that are as similar as possible for every system, as this reduces the diversity of suggestions.

It would now be possible to use the matrices $I, S$ and $P$ to suggest items to a user. To check the effectiveness of the recommender, metrics can be used for evaluation. Numerical metrics include Root Mean Squared Error (RMSE), Mean Absolute Error (MAE) or Normalized Mean Absolute Error (NMAE). These methods check the prediction accuracy by masking items of a user. The actual rating of the user is then compared with the proposed rating of the prediction matrix $P$. Another way to evaluate a recommender is to check the recommender accuracy by comparing the top suggested products with the user's interests. 

As an alternative to the memory-based approach, it is also possible to work model-based in collaborative filtering. Which differs from the basic nearest-neighborhood approach of memory based recommender in that it suggestsproducts from the interaction matrix $I$ based on a model. 

#### 2.2.2 Content based filtering

<img src='./images/flow_content.png'>

In contrast to collaborative filtering recommender systems which make recommendations based on pure interactions (defined by the interaction matrix $I$), the Content Based Recommender also uses other information such as genres of movies, food groups, music styles, etc. These features are additional information about a product and provide information about the preferences of the users. For example, if I listen to mostly classical music, one of my preferences is the classical music genre. With this information, a content-based recommender can suggest pieces of music that overlap with the classical music genre. 

#### 2.2.3 Hybrid recommender system
<img src='./images/flow_hybrid.png' width='80%'>

A hybrid recommender uses more than one method to suggest objects from a wide range of products to a user. It therefore combines several methods. For example, a collaborative filtering based recommender (CBF) can be combined with a content based recommender (CF). By using several methods, it is possible to solve problems that would occur when using a single method. Collaborative filtering has for example the "Early Rater Problem". You cannot create a rating for new products, simply because there is a lack of interaction. Or so-called "Grey Sheeps", users who do not resemble other users can also become a problem. These problems can be solved by integrating another recommender method, for example content-based filtering. ([Hybrid Recommender Systems 2012](http://recommender-systems.org/hybrid-recommender-systems/ ))

### 2.3 Evaluation options
The evaluation of a recommender can be designed using various metrics and heuristics. It is possible to check by mathematical formulas such as Mean Absolute Error (MAE) whether the predicted rating would correspond in principle to what the customer would indicate. For an offline evaluation, various metrics can be introduced in a collaborative filtering recommender with the help of a train/test split. In the test set, customer ratings are masked. A prediction can then be generated with the similarity matrix $S$ based on the train set. The resulting ratings can then be compared with the masked ratings. So here the **prediction accuracy** is measured. There are several metrics for this:

- Mean Absolute Error (MAE), measures the average error margin for a rating. An MAE of 0.5 would indicate that the prediction $p$ is on average 0.5 off from the original rating $r$. (Ekstrand 2011, p. 118)

$$
\frac{1}{n}\sum_{u,i} \vert p_{u,i} - r_{u,i} \vert
$$

- Normalized Mean Absolute Error, measures the average error margin for a rating as well, but normalizes the result. The advantage of normalization using the range of the rating is that different ratings are compared with each other, because the values are normalized to the interval $[0,1]$. $r_{\text{high}}$ and $r_{\text{low}}$ correspond to the highest and lowest possible rating for the recommender. (Ekstrand 2011, p. 118)

$$
\frac{1}{n(r_{\text{high}}-r_{\text{low}})} \sum_{u,i}\vert p_{u,i}-r_{u,i}\vert
$$

- Root Mean Squared Error (RMSE) uses the squared average distance of the prediction $p$ to the rating $r$ from which the root is then taken. This means that larger deviations have more influence than several small ones. (Ekstrand 2011, p. 118)

$$
\sqrt{\frac{1}{n}\sum_{u,i}(p_{u,i}-r_{u,i})^2}
$$

Prediction accuracy measures how well a prediction matches the effective rating. However, the items themselves are not considered. With the **recommender accuracy** a measure can be determined to determine whether the recommender issues relevant products, i.e. whether the items correspond to the customer's preferences. This can be evaluated with the metrics Precision@N and Recall@N.

- Precision@N (N is the number of proposed products) allows to measure how many relevant items $i$ in a proposed set of products $N$ are relevant for the user $u$. This metric is calculated by dividing the number of relevant items $hits_u$ by the number of proposed items $recset_u$. A cross-user precision can be calculated by adding up the precision for each user $u$ and dividing it by the number of users $n$. (Perruchoud 2020, p.3)

$$
P_u = \frac{\vert hits_u \vert}{\vert recset_u \vert}
$$

$$
P = \frac{1}{n} \sum_{u=1}^{n} P_u
$$

- Another measure is Recall@N (N again the number of recommended items). In contrast to Precision@N, it is not the number of relevant products $hits_u$ in the recommended products that is looked at, but the number of relevant, recommended products in relation to the total number of relevant products for the customer $testset_u$. A cross-user recall can be calculated by adding up the recall for each user $u$ and dividing it by the number of users $n$. (Perruchoud 2020, p.3)

$$
R_u = \frac{\vert hits_u \vert }{\vert testset_u \vert}
$$

$$
R = \frac{1}{n} \sum_{u=1}^{n} R_u
$$

---
## 3 Methods
This chapter deals with the above mentioned steps and shows how they were implemented in the system. The methods presented here use simplified algorithms/functions to make them easier to comprehensibly understand. The functions used in the system use additional parameters to adapt to the recommender. You can view these by referring to the source code on the GitHub page of the project.

### 3.1 Data reduction
Building a Recommender system is a data intensive task where a lack of computing power can mean significant limitations. One way to relieve this process is to filter data in advance and thus reduce the number of items for our recommender system. According to our task definition, up to 80 percent of items can be sorted out in advance. Since we want to make recommendations that are as successful as possible and that appeal to the customer, it is best to keep products that are generally popular, while sorting out those that are in less demand.
For our Recommender we have defined three different approachess to reduce the number of items.

#### 3.1.1. Keep top 20% of all products
Here it is counted how often a product has been purchased by all users. The best 20 percent are kept, the other 80 percent are filtered out. Through this method, the most popular items are retained, but the different products aisles are decimated very differently. The advantage of this type of product reduction is that only products that cause the most transactions are retained. However, this can lead to the loss of entire product sections containing products that have a lower re-purchase value, thus reducing the variation of items, which also makes it more difficult to use the recommender to let the user explore product range.

#### 3.1.2. Keep top 20% of products per aisle
This approach is similar to the first one, but the items are first sorted by aisle. After that the top 20 percent products per aisle are kept and the other 80 percent are filtered out. Here, all aisles are reduced to equal parts, thereby maintaining a greater variety of products while still retaining most popular products. 

#### 3.1.3. Keep top 20% according to our own rating
The product reduction options presented above are relatively intuitive. However, it is also possible to create a personal rating for products and filter items based on this ranking. In our dataset, each product contains information about the corresponding department and aisle. With the help of the number of orders, the probability that this product is included in an order can also be taken into account (a priori, support). If we weight the department and aisle based on the number of orders compared to the total number of orders and if we include the information ofthe products we get a rating for each product consisting of the weight of the department, the weight of the aisle and the importance of the product (support). The formula for item $i$ could look like this: 

$$
r_{i} =  \text{weight of department}_i +  \text{weight of aisle}_i + \text{support}(i) 
$$

Note that it is advised to normalize every part of the equation beforehand.

### 3.2 Ratings 

A rating can be created when a user interacts with an item. These ratings can be based on the interaction alone or on a rating that the user has given independently, such as like/dislike or 1-n stars. There are two approaches to obtain this rating. You can let the user rate an item directly. But since the ordinary user is unwilling to give regular feedback, this method can be limited in its effectiveness or if theres no direct rating aviable, it is neccessary to create one based on other aspects like how often or regularly an user interacts with the corresponding item. A rating should also be scaled to the interval 0-1 to be comparable with the evaluation metrics at a later stage. With a memory-based recommender, various ratings may come into question. or our own recommender we have implemented three different ratingsWe use the following:

#### 3.2.1 Binary

This binary rating only shows if an item $i$ was bought by an user $u$.

$${ r(u,i) =
  \begin{cases}
    1 & \quad \text{if } u \text{ has interacted with item } i\\
    0 & \quad \text{otherwise}\\
  \end{cases}}$$

#### 3.2.2 Count
This rating counts how often an item $i$ was bought by an user $u$.

$$r(u,i) = \sum_{i = 1}^n 1 \text{  , for n interactions }$$

#### 3.2.3 Logarithmic hybrid (self-developed) 

This rating is an own approach to create a rating. The following thoughts were made:
- We want to give sufficient weight to the first product purchase. We decided that the first purchase has a weight $\displaystyle \omega = \frac{1}{3}$
    
- Our main weight should be the frequency of a product being in an order. To achieve this, we can take the square-root of the number of orders containing a product  $o$ divided by the total number of orders by the customer $o_{tot}$: $\sqrt{\frac{o}{o_{tot}}}$
    
- Because there are a lot of customers with a low number of orders and related to that a specific uncertainty, we want to weaken the ratings for these customers. To achieve this, we can take the square-root of the total amount of orders for each customer $o_{tot}$ divided by a specific treshold value $m$: $\sqrt{\frac{o_{tot}}{m}}\,\, |\,\, o_{tot} < m\,\,$

- The rating should be a number between 0 and 1

- $\omega$ must be well defined $ 0 < \omega 0.5$, optimally somewhere in the middle

Therefore the following formula has been developed:

$$\displaystyle{ rating(o, o_{tot}) =
  \begin{cases}
    0            & \quad \text{if } o \text{ is } 0\\
    \omega       & \quad \text{if } o \text{ is }1 \\
    \omega + (1 - \omega) \cdot \sqrt{\frac{o}{o_{tot}}}  & \quad \text{if } o \text{ is } > 1 \land \left(  o_{tot} \geq m \right)\\
    \omega + (1 - \omega) \cdot \sqrt{\frac{o}{o_{tot}}} \cdot \sqrt{\frac{o_{tot}}{m}}  & \quad \text{if } o \text{ is } > 1 \land \left(  o_{tot} < m \right)\\
  \end{cases}}$$
    
Where $o$ is the number of orders of the specified product and $o_{tot}$ is the total amount of orders from the corresponding customer $p$.

This approach takes the ratio of each product beeing ordered by a customer and weakens the rating, if to little orders are aviable. We can visualize this formula with the following plot:

In [ ]:
@np.vectorize
def rating(o, o_tot, m = 10, omega = 1/3):
    if o == 0:
        x = 0
    elif o == 1:
        x = omega
    else:
        if o_tot < m:
            w_freq = np.sqrt(o_tot / m)
        else:
            w_freq = 1
        w_prod = np.sqrt(o / o_tot)
        x = omega + (1-omega) * w_prod * w_freq
    return x

o_tot = 50
m = 20
o = np.arange(1, o_tot+1, 1)

values = np.zeros((o_tot,o_tot+1))

for i in range(o_tot):
    for j in range(o_tot+1):
        if i >= j-1:
            values[i,j] = rating(j, i+1, m, omega = 1/3)
        

fig = go.Figure(data=[go.Surface(z=values, x=np.arange(0, o_tot, 1), y=np.arange(0, o_tot+1, 1))])

fig.update_layout(scene = dict(
                    xaxis_title='Orders with Product',
                    yaxis_title='Total orders',
                    zaxis_title='Rating'),
                    )

fig.show()

### 3.3 Interaction matrix $I$
The reduced data now needs to be brought into the shape of a interaction matrix. Here documentet is the evaluation and implementation of the different matrix types.


An interaction or utility matrix lists every interaction between users $u_i$ and items $i_i$. For this reason, the matrix $I$ has the dimensions $u \times i$ . The individual interactions are represented as a numerical values within the matrix and represents a form of rating which is generated by the interaction of the user with the corresponding item. The required transaction matrix can look as follows. Each row represents an interaction with a product. This table records which user (column "user_id") bought which product (column "product_name"). You can already see here that there were users who bought a product more than once. For example, user 3 has already bought product C three times. This enumeration of number of orders can be used as a rating (see 3.2.2) to create an interaction matrix. The transaction data can come from every format. In our case we import the data from a csv in a pandas DataFrame.

In [ ]:
transaction = doc_get_transaction()
transaction

The interaction matrix below has now been created based on the enumerating rating (see 3.2.2) with the transaction list. The interaction matrix now has the dimensions $u \times i$ where $u$ represents the number of users and $i$ the number of products. This matrix should be normalized for further use.

In [ ]:
doc_interaction_count(transaction)

The function `interaction` creates an interaction matrix $I$ with a DataFrame (containing transactions) as input. For small amounts of data, the interaction matrix could easily be created with a [pivot function](https://stackoverflow.com/questions/60776407/create-interaction-matrix-in-python). With larger matrices, however, this can lead to memory problems. An alternative approach, shown here, is to create two lists (`row`, `col`), which in a sparse matrix give the tuple indicating the position of a value. The function is currently designed to match the enumerated rating. With additional parameters other ratings can be implemented. A version adapted to a Modular Recommender is shown here.

In [ ]:
def interaction(df):
    
    # get unique product_names and user_id's
    user_c = CategoricalDtype(sorted(df.user_id.unique()), ordered=True)
    product_name_c = CategoricalDtype(sorted(df.product_name.unique()), ordered=True)
    
    # get (row,col) tuples for sparse matrix
    row = df.user_id.astype(user_c).cat.codes
    col = df.product_name.astype(product_name_c).cat.codes
    
    # create sparse matrix
    val = [1]*len(col)
    sparse_matrix = csr_matrix((val,(row, col)), shape=(user_c.categories.size, product_name_c.categories.size))

    return sparse_matrix

# call function and display as dense matrix
interaction_matrix = interaction(transaction)
interaction_matrix.todense()

As we can see, the function outputs the same matrix as seen above. The return of the function is of type sparse. By calling the `todense()` method we can display the matrix as a dense matrix.

An interaction matrix with a unary rating can be created from the same transaction list. This interaction matrix represents an interaction in binary form (1 = interacted, 0 = not interacted). This also makes it possible to further reduce the transaction list.Double entries are not needed here. The dimension $u \times i$ remains as long as the number of individual users and items does not change.

In [ ]:
doc_interaction_unary(transaction)

#### 3.3.1 Sparse Matrix
As you can see above, a cell is given the value $0$ if there is no interaction between the product $i$ and the customer $u$. An interaction matrix $I$ can therefore largely consist of zeros. Calculating this as a dense matrix can lead to performance problems. To enable a more efficient calculation for later functions, it is therefore useful to display the interaction matrix in sparse format. A sparse matrix stores only the values that differ from zero and records the corresponding indexes (column and row). This drastically reduces the storage overhead.

### 3.4 Similarity matrix $S$
A similarity matrix $S$ is created based on the interaction matrix $I$ and records the similarity $s$ between two users or two items. Since the similarity between all items is calculated, the result is a dense matrix of the dimenson $user \times user$ or $item \times item$. An IBCF calculates the similarities between items, hence the name Item based collaborative filtering. A UBCF (user based collaborative filtering) calculates the similarities between users. An efficient calculation can be done vectorized using cosine similarity, for example. Since the similarity matrix $S$ is a symmetrical matrix ($S_{i,j}=S_{j,i}$ for $i,j = 1,...,n$), only the upper or lower triangular matrix must be calculated ($S_{i,j}$ where $i > j$ for an upper triangular matrix or $i < j$ for a lower triangular matrix).

#### 3.4.1 Cosine Similarity
Cosine similarity measures the similarity between two vectors. To achieve this it calculates the angle between the two vectors. With an intermediate angle of $0°$ (the two vectors point in the same direction) the similarity is $1$, ($cos(0) = 1$). For any other angle the value is less than one. In our example we consider products (columns of the interaction matrix) or users (rows of the interaction matrix) as vectors. In our formula are $i$ and $j$ items which each have a vector consisting of all their ratings $r$. Note: This function can be used for UBCF and IBCF.

$$
s(i,j) = \frac{r_{i}\cdot r_{j}}{\|r_{i}\|_{2}\cdot\|r_{j}\|_{2}} = S_{i,j}
$$

The function `sim_cosine` takes as input the sparse matrix created by the function `interaction` which represents the interaction matrix $I$. The function then calculates the vector norm for each column vector (row vector for UBCF), i.e. for each item. This allows to calculate the similarity $s$ of each item in a for-loop to any other item. The result is a symmetric matrix $S$ that records the similarity of each item to each other. Since the products do not differ to themselves, the diagonal of the matrix is set to the value 1.

In [ ]:
def sim_cosine(df):
    """
    Calculates the cosine similarity between two given vectors
    :param df: sparse matrix with shape (user,item)
    :return s: similarity value between -1 and 1 (1 high correlation, 0 no correlation, -1 high negative correlation)
    """
    
    # initialize empty diagonal matrix
    length = df.shape[1]
    similarity_matrix = np.zeros((length,length), dtype=np.float32) # empty similarity matrix

    # precalculate normalized vector over whole df
    normalized_vectors = norm(df, axis=0)


    for i in np.arange(length):

        # cosine similarity calculation
        a = df[:,i]
        numerator = df.T.dot(a).todense().T # get the dotproduct for vector a and every other vector
        denominator = normalized_vectors*normalized_vectors[i]

        s = numerator/denominator
        similarity_matrix[i,:] = s
        
    return similarity_matrix

sim_cosine(interaction_matrix)

#### 3.4.2 Pearson Correlation
Person Correlation measures the statistical correlation between the ratings of two user's (or items). The formula takes as input two users $u$ and $v$ and calculates a similarity $s$ using the ratings $r$ for every product the two users have in common. It is possible to use pearson correlation to measure the similarity between items as well. But it is said that it yields **worse results** as a cosine similarity for example. The pearson correlation also suffers from problems if there are just a few ratings to compare. A threshold of a number of ratings per user could counteract this problem.

$$
s(u,v) = \frac{\sum_{i\in I_{u}\cap I_{v}}(r_{u,i}-\bar r_{u})(r_{v,i}-\bar r_{v})}{\sqrt{\sum_{i\in I_{u}\cap I_{v}}(r_{u,i}-\bar r_{u})^2} \sqrt{\sum_{i\in I_{u}\cap I_{v}}(r_{v,i}-\bar r_{v})^2}}
$$

#### 3.4.3 Jaccard Similarity
The Jaccard Similarity measures the similarity between two sets $A, B$. The similarity is calculated by dividing the number of elements in the intersection ($\vert{A\cap  B\vert}$) with the size of the union of the two sets ($\vert{A\cup  B\vert}$). The maximum value (total overlap) is $1$. If two sets have no intersection, the value is $0$.

The function looks as follows:

$$
s(A,B) = \frac{\vert{A\cap  B\vert}}{\vert{A\cup  B\vert}}
$$

The function `sim_jaccard` also takes as input the interaction matrix $I$ in sparse format and first creates an empty matrix of the dimension (item, item). In a for-loop, the intersection to other items is calculated for each item. An Intersection is if a user has ordered both items. The denominator is the union of the two items. It counts the total number of users who have ordered one of the two items. The function returns a dense matrix of the dimension $\text{item} \times \text{item}$. A more sophisticated version can be found in the [Plug & Play](https://github.com/roman-studer/fhnw-ds-fs2020-recommender_systems) recommender. There you can use jaccard for an IBCF or a UBCF.

In [ ]:
def sim_jaccard(df):
    """
    Calculates the jaccard similarity between two given sets(vectors) (usful for binary ratings in interaction matrix)
    :param df: sparse matrix with shape (user,item)
    :return s: similarity value between -1 and 1 (1 high correlation, 0 no correlation, -1 high negative correlation)
    """     
    # initalize empty similarity_matrix:
    length = df.shape[1] # length of user vector
    item_length = df.shape[0] # length of item vector

    similarity_matrix = np.zeros((length,length), dtype=np.float32) # empty similarity matrix

    for i in np.arange(length):
        # calculate item overlap between user i and every other user (list of lists)
        numerator = [len([index for index,v,u in zip(np.arange(item_length),df[:,i],df[:,x]) if v > 0 and u > 0 ]) for x in np.arange(length)]
        denominator = [len(np.unique(np.array([index for index,v,u in zip(np.arange(item_length),df[:,i],df[:,x]) if v > 0 or u > 0 ]))) for x in np.arange(length)]
        s = [numerator[i]/denominator[i] for i in np.arange(len(numerator))]
        similarity_matrix[:,i] = s
        
    return similarity_matrix
        
sim_jaccard(interaction_matrix)

### 3.4 Prediction matrix $P$


The value in the prediction matrix $P$, which represents a predicted rating $p$ for an item $i$ from a user $u$, can be calculated differently depending on the recommender. 

- If the recommender uses a rating that is not unary (binary), the following formula can calculate the prediction $p$ for user $u$ and item $i$. To do this, first select a set $S$ consisting of similar products to item $i$ with whom the user has already interacted. The similarity $s$ between the respective items $i$ and $j$ (from $S$) are summed and multiplied by the rating $r_{u,j}$ and then divided by the summed amount of similarities $s(i,j)$.  (Ekstrand 2011, p. 97)

$$
p_{u,i} = \frac{\sum_{j \in S}s(i,j)r_{u,i}}{\sum_{j \in S} \vert s(i,j) \vert}
$$

- As soon as the recommender uses a unary, i. e. binary rating system, the function shown above can no longer keep up. To get a prediction anyway, a pseudo prediction $\tilde{p}_{u,i}$ can be created which adds up the similarities to items from the customer's purchase history. It is also possible to use the average or maximum similarity. (Ekstrand 2011, p. 98)

$$
\tilde{p}_{u,i} = \sum_{j \in I_u} s(i,j)
$$

In [ ]:
def predict(R, mode):
    """
    Generates the predictions for each given user in `R`
    
    :param R: users to predict recommendations, usually the interaction matrix (user/item)
    :param method: method chosen of how `R` was created. 'rating'/'count' will result in equation 2.8, 'binary' will result in equation 2.10
    
    :returns: -scipy.sparse coo_matrix if `method` was 'rating' or 'count'
              - np.array if `method` was 'binary' (float32)
    """
    predictions = None

    if mode == 'rating' or mode == 'count':
        """
        Formula: p_{u,i} = (sum_{j∈S}(s(i,j)*r_{u,j})) / (sum_{j∈S}(abs(s(i,j)))) | S is a set of items similar to i
        
        Equation 2.8 shown in:
        Collaborative Filtering Recommender Systems 2010
        By Michael D. Ekstrand, John T. Riedl and Joseph A. Konstan"""
        
        batchsize = 10000 # tests have shown that this is a good batch size to avoid performance issues
        df_s = pd.read_csv('../data/similar_items/freq_rating_item_similar_objects.csv')
        sim = pickle.load(open('../data/similarity/freq_rating_item_similarity.pkl', 'rb'))
        df_ix = df_s.iloc[:, 2:]
        num_items = int(df_ix.shape[1] / 3)
        s_ix_np = df_ix.iloc[:, num_items:-num_items].to_numpy()
        sim_product = df_s.iloc[:, -num_items:].to_numpy()

        # create sparse similarity matrix where for each column the item_i just contains the k nearest similarities
        # rest is zero for matrix dot product
        col_ix = np.array([s_ix_np.shape[1] * [i] for i in range(s_ix_np.shape[0])]).ravel()
        row_ix = s_ix_np.astype(int).ravel()
        A = np.zeros(sim.shape)
        A[row_ix, col_ix] = 1
        S = A * sim # hadamard product to just keep k similarities
        S = csr_matrix(S)
        
        # perform batchwise predictions
        i_prev = 0
        denominators = 1 / np.sum(np.absolute(sim_product), axis = 1)
        
        for i in range(batchsize, R.shape[0] + batchsize, batchsize):
            # batch prep
            i = min(i, R.shape[0])
            batch = R[i_prev:i]
            
            # numerators
            batch_predictions = batch.dot(S)

            # denominators with hadamard product
            D = np.array([[denominator] * batch_predictions.shape[0] for denominator in denominators]).T
            batch_predictions = batch_predictions.multiply(D)
            
            # append batch to predictions
            if issparse(predictions):
                predictions = vstack([predictions, batch_predictions])
            else:
                predictions = batch_predictions
            
            # update slice index
            i_prev = i
        
    elif mode == 'binary':
        """This mode works only for unary scores.
        
        Formula: p_{u,i} = sum_{j∈I_u}(s(i,j)) | I_u is the user's purchase history
        
        Equation 2.10 shown in:
        Collaborative Filtering Recommender Systems 2010
        By Michael D. Ekstrand, John T. Riedl and Joseph A. Konstan"""

        S = pickle.load(open('../data/similarity/freq_binary_item_similarity.pkl', 'rb'))
        # dot product works because summation of similarities which are in I_u is given if rating is unary
        # and non bought-items are weighted as zero
        I = coo_matrix(R).tocsr()
        predictions = np.float32(I.dot(S)) 
    
    return predictions

### 3.5 Implementation

<img src='./images/flow_recommender.png' width="80%">


The basic structure of the Plug & Play Recommender is divided into four areas. As input we receive a CSV with transactions, respectively orders of products and related information. In a class `DA()` (short for Data Access) the data is prepared for the system and reduced according to one of the mentioned product reduction methods. The class `DA()` contains a function to get one instance of the class (`get_DA`), three methods for product reduction and a function to remove users who have not exceeded a certain number of orders $n$. 

After preparing the data, the class `RS()` (short for recommender system) takes over and creates the functions needed for a Collaborative Based Filtering Recommender to create the various matrices. A function `get_interaction` exists which creates or loads an interaction matrix $I$ on the reduced dataset if it is not yet available. The three mentioned ratings (unary, ordinary, and the logarithmic hybrid) are available as options. With the function `similarity` a similarity matrix $S$ is created on the interaction matrix $I$. Here the above mentioned methods for similarity calculation are available. The function `predict` creates a prediction matrix $P$ and has the above mentioned methods for the creation of predictions $p$ as an option.

Another class `_RecommenderInit()` takes over the function of an interface and assembles the recommender system. Self-made functions (methods discussed in this report) and methods from other packages can be included here. So you can select similarity methods, methods for prediction etc..
Additionally the class `RecEval()` exists which contains methods for evaluating the recommender. Here are the mentioned metrics for Prediction and Recommendation Accuracy. For example, a recommender can be evaluated by calling the function `evaluate()` and providing the appropriate parameters.

All created matrices and CSV's are saved as CSV's or Yaml-files during creation and loaded for further use. With the help of the lazy loader principle, the calculation of matrices can be avoided during a new run. This is useful for testing several evaluation methods one after the other, as it eliminates the need for recalculation. 
If no matrices for a particular recommender exist yet, a waterfall effect will occur. The various steps that require a recommender are carried out in cascade. The prediction function tries to load a suitable prediction matrix $P$. If this matrix does not exist, it calls the appropriate prediction function which creates a matrix. However, this requires a similarity matrix $S$. If this does not exist, the matching matrix $S$ is created. If the function does not have access to the appropriate Interaction Matrix $I$, it will be created. 

If a certain compilation of a recommender has already been run through, the prediction function can directly load an existing matrix $P$ which can then be used for evaluation.

----
## 4 Results
<span style="color:blue">This section will summarize the results of the tested algorithms and methods (including metrics like similarity measures and evaluation methods) and explain them per algorithm. It should also include a description of the comparisons.</span>

### 4.1 Metrics per algorithm
<span style="color:blue">Here the results of the methods used are recorded per algorithm. (if possible in a clear, tabular form)</span>

<span style="color:blue">
Structure:
- Model (IBCF, UBCF, SVD etc.)
- Way of Product Reduction
- Used Similarity Measure (Cosine, Pearson, etc.) 
- Results of evaluation Method. (MAE, NMAE, RSME, Manual)
- Despription (maybe with runtime)</span>

In [ ]:
modes = ['count', 'rating', 'binary']  # modes to create a rating for the interaction matrix 
methods = ['freq', 'aisle', 'rating'] # methods of data reduction
sims = ['cosine'] # jaccard is also implemented but takes to long to compute
recommenders = ['item'] # user not support atm

df_eval = pd.DataFrame(columns=['recommender','reduction','rating','simularity','mae','nmae','rmse','precision','recall'])

for recommender in recommenders:
    for method in tqdm(methods):
        for mode in modes:
            for sim in sims:
                try:
                    test1 = re.evaluate(mode=mode, method=method, sim=sim, recommender=recommender, nr_of_items=20, n=20, k=50, threshold=0.1)
                    
                    test1['recommender'] = recommender
                    test1['rating'] = mode
                    test1['simularity'] = sim
                    test1['reduction'] = method
                    
                    df_eval = df_eval.append(test1, ignore_index=True)
                    
                except:
                    print(f'Recommender with parameters {recommender}, {method}, {mode}, {sim} is not available')

In [ ]:
df_eval

### 4.2 Comparison of the results from diffrent combinations of ratings, similarity functions and prediction functions.

<span style="color:blue">This section brings together all previously recorded results. This allows you to compare them and classify them by effectiveness etc.</span>


<span style="color:blue">After the presentation of the results, a comprehensive description of the results is also included. Here the "winner" or best working combination is announced..</span>

----
## 5 Discussion

<span style="color:blue">Quesitons to answer: What might the answer imply and why does it matter? How does it fit in with what other researchers have found? What are the perspectives for future research?</span>


### 5.1 Description of the results

<span style="color:blue">Here the results of the project work are discussed and evaluated.</span>

<span style="color:blue">How meaningful are our results? This section is dedicated to this question and also partly answers the question whether our system is suitable for further use.</span>


<span style="color:blue">The encountered problems (for expample performance issues) should be documented in this section.</span>

----
## 6 Conclusion and Outlook

<span style='color:red'>Start with short summary and then link to other, similar projects</span>

<span style="color:blue">A review of the project and a short summary of the results.</span>

<span style="color:blue">There are several projects/libraries that contain a similar project (Modular Recommender). It is interesting to make a comparison here. Example: Surprise Recommender library</span>

<span style="color:blue">What information can we take with us? Is it possible to continue the work and build on it?</span>

----
## 7 Sources <a id='source'></a>

“Hybrid Recommender Systems.” 2012. Recommender Systems. http://recommender-systems.org/hybrid-recommender-systems/ (May 15, 2020).

Rocca, Baptiste. 2019. “Introduction to Recommender Systems.” Medium. https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada (May 18, 2020).

Ekstrand, Michael D. 2011. “Collaborative Filtering Recommender Systems.” Foundations and Trends® in Human–Computer Interaction 4(2): 81–173.

“Instacart Market Basket Analysis.” https://kaggle.com/c/instacart-market-basket-analysis (May 22, 2020).

Perruchoud, Daniel. 2020 "Evaluierung von Recommender Systemen"

----
## 8 Glossary